<a href="https://colab.research.google.com/github/Jojocko/NLP-projects-/blob/main/Suite_we_cbow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Datasets NLP/dataset_supplychain.csv')

Mounted at /content/drive


In [2]:
df = df.drop(['client', 'langage', 'reponse'], axis=1)
df.rename(columns={'Sentiment': 'sentiment', 'Commentaire': 'commentaire'}, inplace=True)
df['sentiment'] = df['sentiment'].replace({'__label__POSITIVE': 'positif', '__label__NEGATIVE': 'negatif', '__label__NEUTRAL': 'neutre'})
df['date'] = df['date'].fillna(method="ffill")
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year # pour visualiser par année et non par jour/mois/année

In [3]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('french'))

jugement = {'très', 'extrêmement', 'particulièrement', 'exceptionnellement','tout à fait', 'absolument', 'complètement', 'entièrement', 'parfaitement', 'profondément', 'hautement', 'tout', 'plutôt', 'assez', 'bien', 'bon','vraiment', 'totalement', 'énormément', 'peu', 'moins'}

satisfaction = {'satisfait', 'content', 'heureux', 'ravi', 'enchanté', 'comblé', 'agréable', 'plaisant', 'positif', 'excellent', 'remarquable', 'exceptionnel', 'superbe', 'admirable', 'réjoui', 'gratifiant', 'récompensant', 'conquis', 'impressionné', 'élogieux'}
insatisfaction = {'insatisfait', 'mécontent', 'déçu', 'frustré', 'contrarié', 'désappointé', 'inacceptable', 'problématique', 'inadmissible', 'déplorable', 'lamentable', 'irrité', 'en colère', 'révolté', 'amère', 'négatif', 'critique', 'malheureux', 'peu convaincu', 'regrettable'}
company = {'Fnac', 'fnac', 'Amazon', 'amazon', 'CDiscount', 'cdiscount'}

stop_words.update(jugement, satisfaction, insatisfaction, company)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# Traitement des données pour clustering des commentaires négatifs

import re
import unicodedata
from nltk.tokenize import word_tokenize

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip()) # lower capitalized letters + retire les espaces en début et fin de string (phrases)
    w = re.sub(r"([?.!,¿])", r" \1 ", w) # ajout d'espaces autour des ponctuations pour les séparer des mots. eg: "he is a boy." => "he is a boy .
    w = re.sub(r'[" "]+', " ", w) # remplacement des séquences d'espaces multiples par un seul espace
    w = re.sub(r"[^a-zA-Z?.!]+", " ", w) # suppression de tout caractère qui n'est pas une lettre ou une ponctuation courante
    w = re.sub(r'\b\w{0,2}\b', '', w) # suppression des mots de moins de trois lettres.

    # remove stopword
    mots = word_tokenize(w.strip()) # tokénization en mots individuels
    mots = [mot for mot in mots if mot not in stop_words] # filtrage des mots vides
    return ' '.join(mots).strip() # reconstruction de la phrase sans les mots vides

df.cleaned_lemma = df.cleaned_lemma.apply(lambda x :preprocess_sentence(x))

In [5]:
# Création de l'ensemble de données pour archtecture CBOW

# Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000) # doc limité à 10000 mots
tokenizer.fit_on_texts(df.cleaned_lemma) # application du dic sur les commentaires

word2idx = tokenizer.word_index # stocke le dictionnaire de correspondance entre mots et index
idx2word = tokenizer.index_word # stocke le dictionnaire de correspondance entre index et mots
vocab_size = tokenizer.num_words # stocke la taille du dictionnaire

# Ensemble de données
import numpy as np

def sentenceToData(tokens, WINDOW_SIZE):
    context_size = WINDOW_SIZE // 2
    window = np.concatenate((np.arange(-context_size, 0), np.arange(1, context_size + 1)))
    X, Y = [], []
    for word_index, word in enumerate(tokens):
        if (word_index - context_size >= 0) and (word_index + context_size < len(tokens)):
            context = [tokens[word_index + offset] for offset in window]
            target = word
            X.append(context)
            Y.append(target)
    return X, Y

WINDOW_SIZE = 5  # Fenêtre de contexte autour du mot cible

X, Y = [], []
for review in df['cleaned_lemma'][df['sentiment'] == 'negatif']:
    sentences = review.split(".")  # transforme les commentaires en phrases
    for sentence in sentences:
        word_list = tokenizer.texts_to_sequences([sentence.strip()])[0]  # convertit les phrases en séquence
        if len(word_list) >= WINDOW_SIZE:
            X_temp, Y_temp = sentenceToData(word_list, WINDOW_SIZE)
            X.extend(X_temp)
            Y.extend(Y_temp)

# Convert X and Y to numpy arrays
X = np.array(X).astype(int)

from tensorflow.keras.preprocessing.sequence import pad_sequences

X = pad_sequences(X, padding='post')
Y = np.array(Y).reshape(-1, 1)

print('Shape of X:', X.shape)
print('Shape of Y:', Y.shape)


Shape of X: (764323, 4)
Shape of Y: (764323, 1)


In [6]:
# Architecture CBOW

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, Dropout, GRU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


vocab_size = min(len(word2idx) + 1, tokenizer.num_words + 1)
embedding_dim = 300

optimizer = Adam(learning_rate=0.001)

embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

model = Sequential()
model.add(embedding_layer)
model.add(GRU(units=128, return_sequences=False))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(vocab_size, activation='softmax'))

model.summary()
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         3000300   
                                                                 
 gru (GRU)                   (None, 128)               165120    
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10001)             2570257   
                                                                 
Total params: 5768701 (22.01 MB)
Trainable params: 5768701 (22.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
# Chargement du CBOW

drive.mount('/content/drive')
model.load_weights('/content/drive/My Drive/Datasets NLP/weights.h5')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
from sklearn.metrics.pairwise import cosine_similarity

embeddings = model.layers[0].get_weights()[0]

# Fonction pour trouver les mots les plus similaires dans les embeddings
def find_most_similar(embeddings, word, word_to_index, index_to_word, n=10):
    word_idx = word_to_index[word] # obtenir l'index des mots
    word_embedding = embeddings[word_idx] # obtenir l'embedding du mot cible
    similarities = cosine_similarity([word_embedding], embeddings)[0] # calcul de similarité entre embedding du mot cible et tous les autres embeddings
    most_similar = np.argsort(similarities)[::-1][1:n+1]  # obtenir les indices des mots les plus similaires en ignorant le premier (le mot lui-même)
    similar_words = [(index_to_word[i], similarities[i]) for i in most_similar] # obtenir les mots similaires et leurs scores
    return similar_words

# Correspondance index-mot
index_to_word = {index: word for word, index in tokenizer.word_index.items()}

In [ ]:
# Création de clusters bi-grammes

import numpy as np
from nltk import ngrams
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.text import text_to_word_sequence

negative_comments = df[df['sentiment'] == 'negatif']['cleaned_lemma']

# Générer un mix d'unigrammes, bigrammes, et trigrammes
n_gram_list = []
for comment in negative_comments:
    words = text_to_word_sequence(comment)  # tokenization
    for n in range(1, 4):  # Pour générer unigrammes, bigrammes, et trigrammes
        n_grams = list(ngrams(words, n, pad_left=True, pad_right=True, left_pad_symbol='', right_pad_symbol=''))
        n_gram_list.extend(n_grams)

# Représentation vectorielle pour chaque n-gramme
n_gram_vectors = []
for n_gram in n_gram_list:
    # Filtrer les '' introduits par padding pour calculer l'indice correctement
    filtered_n_gram = [word for word in n_gram if word != '']
    indices = [tokenizer.word_index.get(word, 0) for word in filtered_n_gram]
    # Vérifier si les indices sont valides (< vocab_size)
    if all(idx < vocab_size for idx in indices):
        word_vectors = np.array([embeddings[idx] for idx in indices])
        n_gram_vector = np.mean(word_vectors, axis=0) if len(word_vectors) > 0 else np.zeros(embeddings.shape[1])
        n_gram_vectors.append(n_gram_vector)

# Standardisation des vecteurs de n-grammes
scaler = StandardScaler()
n_gram_vectors_scaled = scaler.fit_transform(n_gram_vectors)


In [ ]:
# PCA pour réduire la dimensionnalité

from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)  # Conserver 95% de la variance explicative
n_gram_vectors_pca = pca.fit_transform(n_gram_vectors_scaled)

# Clustering avec KMeans sur les données réduites
k = 10  # Nombre de clusters
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(n_gram_vectors_pca)
clusters = kmeans.labels_

# Affichage des clusters
cluster_n_grams = {i: [] for i in range(k)}
for i, label in enumerate(clusters):
    n_gram = n_gram_list[i]
    cluster_n_grams[label].append(" ".join(n_gram))

# Afficher les n-grammes pour chaque cluster
for cluster, n_grams in cluster_n_grams.items():
    print(f"Cluster {cluster}:")
    for n_gram in n_grams[:10]:  # Afficher les 10 premiers n-grammes pour concision
        print(n_gram)
    print("\n---\n")

In [ ]:
# Score de silhouette pour aider à identifier un nombre optimal de clusters pour votre analyse, en fonction du score de cahque cluster

# Calcul long d'où échantillonnage des données

from sklearn.metrics import silhouette_score

# Calculer le score de silhouette avec un échantillon limité
score = silhouette_score(n_gram_vectors_pca, kmeans.labels_, sample_size=50000)
print(f"Score de silhouette (limité): {score}")


In [ ]:
# Test des clusters formés

ref_df = pd.read_csv('/content/drive/My Drive/Datasets NLP/ecommerce_plaintes.csv')

# Accès aux commentaires
reference_sentences = ref_df['commentaire'].tolist()

# Tokenisation et conversion en embeddings pour les phrases de référence
ref_n_gram_vectors = []

for sentence in reference_sentences:
    words = text_to_word_sequence(sentence)
    n_grams = list(ngrams(words, n))  # n étant bigramme
    sentence_vectors = []
    for n_gram in n_grams:
        indices = [tokenizer.word_index.get(word, 0) for word in n_gram]
        if all(idx < vocab_size for idx in indices):
            word_vectors = embeddings[indices]
            n_gram_vector = np.mean(word_vectors, axis=0)
            sentence_vectors.append(n_gram_vector)
    # Moyenne des embeddings des n-grammes pour obtenir un vecteur par phrase
    if sentence_vectors:
        ref_n_gram_vectors.append(np.mean(sentence_vectors, axis=0))

ref_n_gram_vectors = np.array(ref_n_gram_vectors)


In [ ]:
# Même objet PCA que pour les données d'entraînement

ref_n_gram_vectors_pca = pca.transform(ref_n_gram_vectors)

In [ ]:
# Calcul de similarité

from sklearn.metrics.pairwise import cosine_similarity

# Calculer la similarité de chaque phrase de référence avec les centroïdes des clusters
similarities = cosine_similarity(ref_n_gram_vectors_pca, kmeans.cluster_centers_)

# Pour chaque phrase, identification du cluster le plus proche
closest_clusters = np.argmax(similarities, axis=1)

for i, cluster_num in enumerate(closest_clusters):
    print(f"Phrase '{reference_sentences[i]}' est la plus proche du cluster {cluster_num}")


In [ ]:
from joblib import dump, load

# Sauvegarder le modèle
dump(kmeans, 'kmeans_model.joblib')

# Sauvegarder les vecteurs d'embedding
dump(n_gram_vectors_scaled, 'n_gram_vectors_scaled.joblib')

In [ ]:
# Later charger les modèles:

# Kmeans :
# kmeans = load('kmeans_model.joblib')

# Vecteurs d'embedding
# n_gram_vectors_scaled = load('n_gram_vectors_scaled.joblib')

# Utiliser le modèle chargé pour prédire les clusters d'un nouvel ensemble de données, ou réutiliser les labels déjà calculés
# clusters = kmeans.labels_
# ou kmeans.predict(nouvel_ensemble_de_données_mis_à_l'échelle)